# ICT3104 Project Team 12

## Setup
These libraries are required to run the tool.

In [ ]:
# Import libraries
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, Video

## Data Exploration
This section lets you select and view the videos from the data/input_files/training_videos folder, allowing you to verify if you have chosen the correct video.
To view the videos, select an option from the dropdown video and wait for the video to be displayed.

In [ ]:
# Populate dropdown with files in folder
folder_path = "./ict3104-team12-2023/data/input_files/training_videos/"
video_options = os.listdir(folder_path)

dropdown = widgets.Dropdown(options=video_options)

# Define a function to update and display the video based on the selected option
def update_video(change):
    selected_option = change['new']

    # Clear the previous video output before displaying the new one   
    clear_output(wait=True)
    display(dropdown)

    # Check if file is in valid mp4 format
    if ".mp4" in selected_option:
        video_path = f"{folder_path}{selected_option}"
        video_display = Video(video_path, width=400, height=400, embed=True)
        display(video_display)
    else:
        print("Please choose a file with .mp4 format.")

# Register the function to be called when the dropdown value changes
dropdown.observe(update_video, names='value')

# Display the initial video based on the default selected option
initial_option = dropdown.value
update_video({'new': initial_option})

## Inference

## Training

## Testing